
$\newcommand{\mat}[1]{\boldsymbol {#1}}$
$\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}$
$\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}$
$\newcommand{\vec}[1]{\boldsymbol {#1}}$
$\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}$
$\newcommand{\rvar}[1]{\mathrm {#1}}$
$\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}$
$\newcommand{\diag}{\mathop{\mathrm {diag}}}$
$\newcommand{\set}[1]{\mathbb {#1}}$
$\newcommand{\cset}[1]{\mathcal{#1}}$
$\newcommand{\norm}[1]{\left\lVert#1\right\rVert}$
$\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}$
$\newcommand{\bb}[1]{\boldsymbol{#1}}$
$\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}$
$\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}$
$\newcommand{\given}[]{\,\middle\vert\,}$
$\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}$
$\newcommand{\grad}[]{\nabla}$
$\newcommand{\norm}[1]{\left\lVert#1\right\rVert} $

# Part 4: Summary Questions
<a id=part4></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.

2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.

### CNNs Answers:
1. Receptive fields are defined portion of space or spatial construct containing units that provide input to a set of
units within a corresponding layer. The receptive field is defined by the filter size of a layer within a convolution neural network. $\\$
For example if we have a single convolutional layer with a kernel size of 3×3 than every cell in the output of that layer will be affected by a 3×3 region in the layers input (Except for the cells near the edge of input).
2. After some research we did on the subject, we found out that there are a few ways to *increase* the receptive field: $\\$
a. Add more convolutional layers (make the network deeper) $\\$
b. Add pooling layers or higher stride convolutions (sub-sampling) $\\$
c. Use dilated convolutions $\\$
In addition, we found a way to calculate the recptive field of the model. Let's say we have 2 convoutional laters $f1, f2$,
with kernel size $k$, stride $s$ and receptive field $r$, then:
$r1 = s2 \cdot r2 + (k2 - s2)$. So we can say that the kernel size and receptive field of the previous layer also affect the result. $\\$
In summary, more layers will result us a linear increase in the RF without a loss of resolution (like pooling).
    Pooling layers and dilation enable an exponential growth in the RF but offer a trade off between
    no resolution lost (dilation) and invariance to small translations (pooling)
3. Let's split the calculations layer by layer: $\\$
a. **The first conv**: We use a 3x3 kernel and padding of 1. As we learned, this means that the output shape will be the same
as the input. The RF of the inner cells (they are not affected by padding) has the size of 3x3. $\\$
b. **ReLU**: Each calculation depends on a single cell, which means that there is no affect on the size of the RF which remains 3x3. $\\$
c. **Pooling no.1**: We use a 2x2 kernel $\Rightarrow$ the new RF will be: $(3+(2-1)) \times (3+(2-1)) = 4 \times 4$.
In addition, this layer has a stride of 2, so the increase in the RF in all the upcoming layers will be doubled. $\\$
d. **The second conv**: Now we have a 5x5 kernel. This means that every cell in the output will have a RF of $(4+2 \cdot (5-1) \times (4+2 \cdot (5-1) = 12 \times 12$ . $\\$
Notice that like the pooling layer above, there is a use of stride of 2, which will result in all RFs in the following layers doubling again. $\\$
e. **ReLU**: As mentioned above, this layer has no affect on the RF. $\\$
f. **Pooling no.2**: A kernel of size 2 is being used which means that the new RF size is: $(12 + 2 \cdot 2 \cdot (2-1)) \times (12 + 2 \cdot 2 \cdot (2-1)) = 16 \times 16$ . $\\$
Since this layer has a stride of 2, the increase in the RF in all the upcoming layers will be doubled. $\\$
g. **The third conv**: In this layer we have a 7x7 kernel and dilation of 2, which will result (again) in doubling the RF.
As a result, every cell in the output layer will have a RF of: $(16 + 2 \cdot 2 \cdot 2 \cdot 2 \cdot (7-1)) \times (16 + 2 \cdot 2 \cdot 2 \cdot 2 \cdot (7-1)) = 112 \times 112$ . $\\$
h. **ReLU**: As mentioned above, no effect. $\\$
To sum up, the RF is $112 \times 112$ . $\\$
4. We got different results is because we added $x$ to the output. This is like increasing the weight in the center of the
kernel by 1. If we want the new model to have equal weights to the original model, we would need to decrease the weight at the
center of the kernel, and increase the weights at the rest of the kernel. This causes the weights to look completely different. $\\$

### Dropout

1. **True or false**: dropout must be placed only after the activation function.

2. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

### Dropout Answers:
1. **false**. Dropout can also be placed before the activation function. Typically, dropout is applied after the activation function
but, in the case of ReLU, it sometimes makes sense to use dropout before it. $\\$
2. Assume that the value of a certain activation function is $a$ . We will denote $D$ as dropout function with probability $p$ .
Therefore: $\mathbb{E}[D(a)] = p \cdot 0 + (1-p) \cdot \frac{1}{1-p} \cdot a = a$ . This proves that the expected value of the
dropout's output equals to the input. $\\$

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [2]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H),
        nn.Sigmoid(),
    ]*N,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
    1. In a model using exclusively ReLU activations, there can be no vanishing gradients.
    1. The gradient of ReLU is linear with its input when the input is positive.
    1. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.

### Losses and Activation functions Answers:
1. We would not use L2 loss. L2 Loss Function is used to minimize the error which is the sum of the all the squared
differences between the true value and the predicted value. In our case, there are only 2 possible values: 1 and 0. $\\$
Instead we will use the Binary Cross Entropy loss function a.k.a the Log loss $\\$
Let's examine this example: assume that the correct label is 1. In the case of our model predicting, 0.0001, the BCE loss would
be at around 9.2, and the L2 loss would be 0.9999. That will result in the model using BCE loss function to learn much faster. $\\$
2. The most likely cause is that he have a problem of vanishing gradient. As we learned from class, when using the sigmoid
activation function, there is a chance to encounter the vanishing gradient problem, because the values of the derivative
of sigmoid are between 0 to 0.25. Those are small values which may lead to the gradient becoming zero during backpropagation.$\\$
3. The friend is not correct. As we know, for tanh there is also a chance for us to encounter the vanishing gradient problem.
If our friend would have listend in class, he would have suggested us to use ReLU, which is "immune" to the vanishing gradient problem.$\\$
4. i. **true**. ReLU has gradient 1 when input > 0, and zero otherwise. Thus, multiplying a bunch of ReLU derivatives together in the backprop equations has the nice property of being either 1 or 0. There is no “vanishing” or “diminishing” of the gradient. The gradient travels to the bottom layers either as is or it becomes exactly 0 on the way. $\\$
ii. **false**. As stated above, the gradient of ReLU is a const 1 when the input is positive. ReLU function is linear
with its input when the input is positive but not the derivative.$\\$
iii. **true**. As we know, ReLU has a value of zero when the input is negative, so if the majority of the input or in
the worst case scenario, the whole input is negative, we will get a constant function of zero (in the worst case). Let's
examine a bit more this situation when most of the inputs to these ReLU neurons are in the negative range. $\\$
When most of these neurons return output zero, the gradients fail to flow during backpropagation, and the weights are not updated. Ultimately a large part of the network becomes inactive, and it is unable to learn further.

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

2. Regarding SGD and GD:
    1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
    2. In what cases can GD not be used at all?

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

4. For each of the following statements, state whether they're **true or false** and explain why.
    1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
    1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
    1. SGD is less likely to get stuck in local minima, compared to GD.
    1. Training  with SGD requires more memory than with GD.
    1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
    1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
    1. **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
    1. Explain the concepts of "vanishing gradients", and "exploding gradients".
    2. How can each of these problems be caused by increased depth?
    3. Provide a numerical example demonstrating each.
    4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?

### Optimization Answers:
1. In both gradient descent (GD) and stochastic gradient descent (SGD), we update a set of parameters in an iterative manner to minimize
some error function. The difference is that in GD, we run through all the samples in the training set to do a single update,
 while in SGD, we use only one training sample. There is also a version of SGD where instead of one sample, we use a subset
 of the training set which is called Minibatch SGD.$\\$
2. i. First of all, by using GD you need to store all the dataset in memory instead of storing a singular datapoint or subset of the dataset.$\\$
Secondly, By using SGD instead of GD we get dynamic error surface. Because we compute an error surface for each batch and not for all the dataset. By
doing so we might help the optimizer to get out of flat regions or sharp local minima (because they might not exist in the error surface of subsequent
batches).$\\$
Another advantage of using SGD is that it is faster than regular GD and will converge much faster compared to GD, because
in each iteration we don't have to look at the whole training set which saves us time. $\\$
ii. In some cases, GD can't be used. For example, cases where the data is arranged in a way that it poses a non-convex optimization problem,
this is because gradient descent only works for problems which have a well defined convex optimization problem. Another
example is when we don't have enough space to store all the dataset in memory, in this case, the network will fail. $\\$
3. We expect the number of iterations to decrease, because by increasing the size of each batch, every update will take into account more examples from the data
and as a result, will be more accurate. $\\$
4. i. **false**. SGD chooses a random sample in every iteration, which means there is no guarantee that we will use every sample. $\\$
ii. **false**. The gradients of SGD have more variance, because they represent only a part of the real distribution of data.
On the other hand, GD represents a much bigger group of samples and therefore its gradients' variance in smaller. $\\$
iii. **true**. In stochastic gradient descent the parameters are estimated for every observation, as opposed to the whole
sample in regular gradient descent. This is what gives it a lot of randomness. The path of stochastic gradient descent
wanders over more places, and thus is more likely to "jump out" of a local minimum, and find a global minimum. $\\$
iv. **false**. As we mentioned already, GD requires more memory because we need to store the whole dataset. $\\$
v. **false**. Both can converge to a global or to a local minima, however the chance for SGD to converge to global minima
is higher. $\\$
vi. **false**. Newton's method converges quicker in the ravine as it uses the Hessian to find a direct path to the minimum.
SGD with momentum should handle this problem quite well, as on average it will take a step in the correct direction of the minimum, but it
might take more iterations to reach the bottom of the surface. $\\$
5. **false**. As was mentioned in the tutorial, it doesn't matter how we solve the inner optimization problem in the forward pass
(quote: "... either with some solver or even a closed-form expression ..."). It is however, important to calculate the derivative
using the method which was described in the tutorial. $\\$
Let's assume we have a solution for the inner optimization problem and call that solution $i$ .
In addition, let's call the input to the said problem $x$ . Now we can calculate: $K=∇_{ii}^2f(i,x), R=∇_{ix}^2f(i,x)$ . $\\$
Now we can calculate $δu=K^{-1}δi$ and then $δx=R^{⊤}δu$ . As we can see, the calculation method has no effect on the result. $\\$
6. i. Vanishing Gradient occurs when the derivative or slope will get smaller and smaller as we go backward with every layer during backpropagation.
When weights update is very small or exponential small, the training time becomes much longer, and in the worst case, this may completely stop the neural network training. $\\$
Exploding gradient occurs when the derivatives or slope will get larger and larger as we go backward with every layer
during backpropagation. This situation is the exact opposite of the vanishing gradients. $\\$
ii. Vanishing gradient occurs typically with the sigmoid and tanh activation functions because their derivatives are between
0 to 0.25 and 0 to 1 respectively. Therefore, the updated weight values are small, and the new weight values are very similar to the old weight values.
As we add more and more layers, the chance for the values of the derivatives to become zero during backpropagation increases. $\\$
In contrast to vanishing gradients, exploding gradients problem happens not because of the activation function, but because
of the weights. If the weights are high, it will cause the value of the derivative to also increase, which will result in the
new updated weights being much more varied from the previous weights, and the gradient to never converge. $\\$
iii. Vanishing Gradients: Let's say we have a model with a linear layer and then a big number of sigmoid activation functions.
As we know, the maximum value of a sigmoid derivative is 0.25, which means that at the best case, after calculating the gradient of the first sigmoid, it will be
multiplied by $0.25^{N-1}$ , where $N$ is the number of sigmoid layers. This will decrease to 0 very (very) fast. $\\$
Exploding Gradients: Let's say we have a model with a large number of linear layers and a ReLU activation function between every pair of linear layers.
In addition, let's assume that the input to the ReLu function is always positive, which will result in the gradient of ReLU to be always 1.
Now if the geometric mean of the gradients of the linear layers is a number $x > 1$ (for example 2), this will result in
an exponential increase in the values of the gradients. $\\$
iv. We can try and find the cause for the problem. For example, if we'll find out that: $\\$
a. The parameters of the higher layers change significantly whereas the parameters of lower layers do not change much (or not at all) $\\$
b. The model weights become 0 during training. $\\$
c. The model learns very slowly and perhaps the training stagnates at a very early stage just after a few iterations $\\$
Then, the problem is vanishing gradient. $\\$
If however, we find out that: $\\$
a. There is an exponential growth in the model parameters. $\\$
b. The model weights become NaN during training. $\\$
c. The model experiences  avalanche learning. $\\$
Then, the problem is exploding gradient.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $\hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2$
  $\\$Your wish to minimize the in-sample loss function is defined as

  $L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)$

  Where the pointwise loss is binary cross-entropy:

  $\ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})$

  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.

2. Given the following code snippet, implement the custom backward function `part4_affine_backward` in `hw4/answers.py` so that it passes the `assert`s.

In [3]:
from torch.autograd import Function

from hw4.answers import part4_affine_backward

N, d_in, d_out = 100, 11, 7
dtype = torch.float64
X = torch.rand(N, d_in, dtype=dtype)
W = torch.rand(d_out, d_in, requires_grad=True, dtype=dtype)
b = torch.rand(d_out, requires_grad=True, dtype=dtype)

def affine(X, W, b):
    return 0.5 * X @ W.T + b

class AffineLayerFunction(Function):
    @staticmethod
    def forward(ctx, X, W, b):
        result = affine(X, W, b)
        ctx.save_for_backward(X, W, b)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        return part4_affine_backward(ctx, grad_output)

l1 = torch.sum(AffineLayerFunction.apply(X, W, b))
l1.backward()
W_grad1 = W.grad
b_grad1 = b.grad

l2 = torch.sum(affine(X, W, b))
W.grad = b.grad = None
l2.backward()
W_grad2 = W.grad
b_grad2 = b.grad

assert torch.allclose(W_grad1, W_grad2)
assert torch.allclose(b_grad1, b_grad2)

### Backpropagation Answers:
1. We will split the loss equation into the following parts: $\\$
$$z_{1}=W_{1}x_{i}$$ $\\$
$$z_{2}=z_{1}+b_{1}$$ $\\$
$$z_{3}=\varphi(z_{2})$$ $\\$
$$z_{4}=W_{2}z_{3}$$ $\\$
$$\hat{y}_{i}=z_{4}+b_{2}$$ $\\$
$$L_{S,i}=(-y_{i}log(\hat{y}_{i})-(1-y_{i})log(1-\hat{y}_{i}))$$ $\\$
$$L_{S}=\frac{1}{N}\sum_{i=1}^{N}(L_{S,i})+\frac{\lambda}{2}(||W_{1}||_{F}^{2}+||W_{2}||_{F}^{2})$$ $\\$

First, we will calculate the gradient w.r.t $L_{S,i}$, then we will calculate the gradient w.r.t the complete loss.
As was shown in the tutorial and the lecture:

$$\frac{\partial L_{S,i}}{\partial\hat{y}_{i}}=-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}$$ $\\$
$$\frac{\partial L_{S,i}}{\partial b_{2}}=\frac{\partial\hat{y}_{i}}{\partial b_{2}}\frac{\partial L_{S,i}}{\partial\hat{y}_{i}}=\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial z_{4}}=\frac{\partial\hat{y}_{i}}{\partial z_{4}}\frac{\partial L_{S,i}}{\partial\hat{y}_{i}}=\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial W_{2}}=\frac{\partial z_{4}}{\partial W_{2}}\frac{\partial L_{S,i}}{\partial z_{4}}=\left(\varphi(W_{1}x_{i}+b_{1})\right)^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial z_{3}}=\frac{\partial z_{4}}{\partial z_{3}}\frac{\partial L_{S,i}}{\partial z_{4}}=W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial z_{2}}=\frac{\partial z_{3}}{\partial z_{2}}\frac{\partial L_{S,i}}{\partial z_{3}}=\begin{pmatrix}\ddots & 0 & 0\\
0 & \varphi'\left(\left(W_{1}x_{i}+b_{1}\right)_{k}\right) & 0\\
0 & 0 & \ddots
\end{pmatrix}W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial b_{1}}=\frac{\partial z_{2}}{\partial b_{1}}\frac{\partial L_{S,i}}{\partial z_{2}}=\begin{pmatrix}\vdots\\
\varphi'\left(W_{1}x_{i}+b_{1}\right)_{k}\\
\vdots
\end{pmatrix}W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial z_{1}}=\frac{\partial z_{2}}{\partial z_{1}}\frac{\partial L_{S,i}}{\partial z_{2}}=\begin{pmatrix}\ddots & 0 & 0\\
0 & \varphi'\left(W_{1}x_{i}+b_{1}\right)_{k} & 0\\
0 & 0 & \ddots
\end{pmatrix}W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$ $\\$
$$\frac{\partial L_{S,i}}{\partial W_{1}}=\frac{\partial z_{1}}{\partial W_{1}}\frac{\partial L_{S,i}}{\partial z_{1}}=\begin{pmatrix}\ddots & 0 & 0\\
0 & \varphi'\left(W_{1}x_{i}+b_{1}\right)_{k} & 0\\
0 & 0 & \ddots
\end{pmatrix}W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)x_{i}^{T}$$ $\\$
$$\frac{\partial L_{S,i}}{\partial x_{i}}=\frac{\partial z_{1}}{\partial x_{i}}\frac{\partial L_{S,i}}{\partial z_{1}}=W_{1}^{T}\begin{pmatrix}\ddots & 0 & 0\\
0 & \varphi'\left(W_{1}x_{i}+b_{1}\right)_{k} & 0\\
0 & 0 & \ddots
\end{pmatrix}W_{2}^{T}\left(-\frac{y_{i}}{\hat{y}_{i}}+\frac{1-y_{i}}{1-\hat{y}_{i}}\right)$$

Finally, we will calculate the gradient w.r.t the complete loss function: $\\$
$$\frac{\partial L_{S}}{\partial W_{2}}=\frac{1}{N}\sum_{i=1}^{N}(\frac{\partial L_{S,i}}{\partial W_{2}})+\lambda W_{2}$$ $\\$
$$\frac{\partial L_{S}}{\partial b_{1}}=\frac{1}{N}\sum_{i=1}^{N}(\frac{\partial L_{S,i}}{\partial b_{1}})$$ $\\$
$$\frac{\partial L_{S}}{\partial W_{2}}=\frac{1}{N}\sum_{i=1}^{N}(\frac{\partial L_{S,i}}{\partial W_{1}})+\lambda W_{1}$$ $\\$
$$\frac{\partial L_{S}}{\partial x}=\frac{1}{N}\sum_{i=1}^{N}(\frac{\partial L_{S,i}}{\partial x_{i}})$$ $\\$

### Sequence models

1. Regarding word embeddings:
    1. Explain this term and why it's used in the context of a language model.
    1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?

2. Considering the following snippet, explain:
    1. What does `Y` contain? why this output shape?
    2. How you would implement `nn.Embedding` yourself using only torch tensors. 

In [4]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of $S$: State whether the following sentences are **true or false**, and explain.
    1. TBPTT uses a modified version of the backpropagation algorithm.
    2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length $S$.
    3. TBPTT allows the model to learn relations between input that are at most $S$ timesteps apart.

### Sequence models Answers:
1. i. Word embedding is a learned representation for text where words that have the same meaning have a similar representation.
For example: person and human would have a similar respresentation, where peace and war would not. Let's also remind ourselves
the example from class, where we could do what is called "arithmetics of word embedding". For example:
King - Man + Woman = Queen. $\\$ Word embedding is used in language models because it enables the model to learn words and sentences which
are semantically similar by using basic algebra and calculus. $\\$
ii. It is possible, by using for example the method Bag-of-Words. Bag-of-Words is a simplifying representation used in
natural language processing. In this method, a text (such as a sentence or a document) is represented as the bag (multiset) of its words, disregarding grammar and even word order but keeping multiplicity.$\\$
In case we do implement this, the model will struggle to understand the connection between words that are not close. For
example, the model will have a hard time understanding the sentence "The food wasn't great but I still liked it", which is
hard to infer without sequential context. $\\$
2. $Y$ contains the embedding of $X$ . $X$ is a tensor of shape $(5,6,7,8)$ . As we know, embedding is a mapping of the indices,
which means that $Y$ returns for each of $X$'s indices the corresponding embedding. Since each embedding is a vector is size $42000$ ,
we can deduce that $Y$ is a vector of shape $((5,6,7,8),42000)$ . $\\$
3. i. **true**. TBPTT is a modified version of BPTT training algorithm. It needs to track the depth it reached so far, so
that when the last $S$ tokens where propagated, the algorithm will stop. $\\$
ii. **false**. Although this method works for a certain aspect, it is very limiting, and a much better solution is implementing the
modified algorithm stated in the previous section of the question. $\\$
iii. **false**. The hidden states allow the gradients to pass through sequences which are of length greater than $S$ . $\\$

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
    1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


### Attention Answers:
1. The attention mechanism creates a weighted average of the encoder's output, named keys and values which are based on the
decoder's hidden state, named query. This allows the mechanism to produce context for the decoder, which helps the decoder
to not be entirely dependent on its last hidden state to convey the whole context, because now the decoder can
look at other parts of the source sequence. $\\$
2. The reason to use a transformer is to allow the decoder to understand the context from the encoder's hidden states. As stated
in the question, now all the inputs to the transformer are from the encoder, which will make it unable to correlate the context
between the encoder and the decoder. This is why, we excpect the new model to behave like a model with no attention at all.
Maybe the transformer will be able to find out some relations between the decoder's/encoder's hidden states. $\\$


### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

    1. Images reconstructed by the model during training ($x\to z \to x'$)?
    2. Images generated by the model ($z \to x'$)?

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
    1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
    2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
    3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.

3. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
    1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
    2. It's crucial to backpropagate into the generator when training the discriminator.
    3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
    4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
     5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.

### Unsupervised learning Answers:
1st question answers:
1. Images reconstructed by the model will not be affected by it, because that the purpose of the KL-divergence term is to regularize the distribution we are trying to find while the reconstruction term serves as a regression model trying to produce an identity mapping. So without the KL-divergence term our model will try to reconstruct the original images and will create similar outputs.
2. The images generated by the model will be very bad. The purpose of KL-divergence is to make the feature space and
latent space be mapped as perfectly as we can so that the decoder would be able to translate every sample from $q_\alpha$
to a clear image. Without it, we will have a normal distribution and as a result, the images that the model will make
will be just some normally distributed noise, very similar to the images we saw when first training the model.

2nd question answers:
1. False. The latent-space distribution generated by the model given a specific input is $\mathcal{N}(μ_x,\sigma_x^2I) $
2. False. The encoder samples from a normal vector to approximate the posterior distribution using the reparametrization trick, we will get different encoding for the same input, and it will lead to different reconstructions.
3. True. Because we don't know the real distribution we approximate it with ELBO bound as we seen in the lectures.

3rd question answers:
1. False. We want both losses to be low, because if the discriminator loss is high it means that the generator can easily fool him, but it does not mean that the generator is creating good images.
3. False. when training the discriminator we don't want to backpropogate to the generator because it creates a feedback loop which can lead to the generator learning how to fool the discriminator and learning how to create quality images.
4. True. The input to the generator is a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
5. True. Training the discriminator a few epochs first will allow it to learn better what is a real image and this will give more meaningful results to the generator in the first epochs which will lead to more realistic images.
6. False. When the discriminator accuracy is 50% it means that he does not know what is a real image and fake one, so his results are meaningless. But if we use those results to train the generator it could lead to lesser quality images in the generator.


### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$
    1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
    1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?

### Graph Neural Networks Answers:
1. i. The layer above does a spatial convolution and applies a non-linearity activation function. $\\$
$\Rightarrow$ Every row $r$ in $Y$ will contain the features extracted from vertex number $r$ in the graph. $\\$
ii. In the matrix $Y$ , each row $i$ represents the effect of all the neighbours of the $i^{th}$ vertex on it, and the
column $j$ represents all the effects of the $j^{th}$ vertex on its neighbours. Since both row and col number 5 are zeros,
it is like the $5^{th}$ vertex isn't affected by anyone and has no effect on anything. Meaning, it is like this vertex doesn't
exist in the graph. In conclusion, the vertex won't appear on the graph. $\\$
2. We would define the RF of a node as the group of nodes that might affect its output. For example, in the spatial convolution
mentioned in the question, the RF will be all the nodes which are within distance $q$ to the node. $\\$